# Workshop : รถขับเคลื่อนอัตโนมัติด้วยปัญญาประดิษฐ์ (AI)
## Chiang Mai Maker Party 07-08 December 2019

## Example 3 : Object Detection

### การเก็บข้อมูลรูปภาพ (Collect Data)

การทำ Object Detection จะมีขั้นตอนดังต่อไปนี้
- เก็บรูปภาพของวัตถุที่ต้องการจำแนก (Dataset)
- นำรูปภาพมาเฉลย (Label Dataset)
- เทรนโมเดล (Train)
- ทดสอบโมเดล (Evaluate)


### เรียกใช้งานไลบรารี่
- jetcam
- display
- widgets
- traitlets

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# init jetcam camera
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera

# set camera size to 224 x 224 30 FPS
camera = CSICamera(width=224, height=224, capture_fps=30)
camera.running = True

from jetbot import Robot
robot = Robot()

from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

print("Jetbot Start........")

### แสดงภาพจากกล้อง

In [ ]:
image_widget = widgets.Image(format='jpeg', width=200, height=200)

x_slider = 0
y_slider = 0
       
def display_xy(camera_image):
    image = np.copy(camera_image)
    print(image.shape)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)

### เรียกใช้งาน Gamepad

In [ ]:
from inputs import get_gamepad
import time
import sys
import numpy as np
import threading


motor_left_value = 0
motor_right_value = 0

x_val = 0
y_val = 0

### สร้าง thread เพื่ออ่านค่าจาก Gamepad

In [ ]:
import threading

img_count = 0

def thread_function(name):
    while True:
        motor_r = .3 * y_val - .15 * x_val
        motor_l = .3 * y_val + .15 * x_val

        robot.left_motor.value = motor_l
        robot.right_motor.value = motor_r

x = threading.Thread(target=thread_function, args=(2,))
x.start()

def throttle(state):
    global y_val
    global motor_left_value, motor_right_value
    y_val = np.interp(state, [0, 255], [0.9, -0.9])

def steering(state):
    global x_val
    global left_factor, right_factor
    global motor_left_value, motor_right_value
    x_val = np.interp(state, [0, 255], [-0.9, 0.9])

### แก้ไขชื่อไฟล์ที่จะทำการบันทึกรูปภาพ

file = "ชื่อ path ที่ต้องการบันทึก/{0}{1}.jpg".format("ชื่อวัตถุ", img_count)

file = "data/{0}{1}.jpg".format("human", img_count)

สามารถแก้ไขคำว่า car เป็นชื่อของวัตถุที่ต้องการ Detect

In [ ]:
def read_joystick(state):
    global img_count
    img_count = img_count + 1
    file = "data/images/{0}{1}.jpg".format("human", img_count)
    print(file)
    cv2.imwrite(file, camera.value)
    time.sleep(.5)
        
event_lut = {
    'ABS_Y': throttle,
    'ABS_Z': steering,
    'BTN_TL':read_joystick,
}

### เริ่มต้นการทำงานโปรแกรม

In [ ]:
def main():
    events = get_gamepad()
    for event in events:
        call = event_lut.get(event.code)
        if callable(call):
            call(event.state)

if __name__ == "__main__":
    try:
        while True:
            main()
    except KeyboardInterrupt:
        robot.stop()
        x.stop()
        print("Bye!")